In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 102kB 30.9MB/s ta 0:00:01


In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test1.csv')
test = test.iloc[:, 1:]
train = train.iloc[:, 1:]
train.head()
features=train.columns.tolist()
features.remove('label')
# 对osv进行数据清洗
def osv_trans(x):
    x = str(x).replace('Android_', '').replace('Android ', '').replace('W', '')
    if str(x).find('.')>0:
        temp_index1 = x.find('.')
        if x.find(' ')>0:
            temp_index2 = x.find(' ')
        else:
            temp_index2 = len(x)

        if x.find('-')>0:
            temp_index2 = x.find('-')
            
        result = x[0:temp_index1] + '.' + x[temp_index1+1:temp_index2].replace('.', '')
        try:
            return float(result)
        except:
            print(x+'#########')
            return 0
    try:
        return float(x)
    except:
        print(x+'#########')
        return 0
    
#train['osv'] => LabelEncoder ?
# 采用众数，进行缺失值的填充
train['osv'].fillna('8.1.0', inplace=True)
# 数据清洗
train['osv'] = train['osv'].apply(osv_trans)

# 采用众数，进行缺失值的填充
test['osv'].fillna('8.1.0', inplace=True)
# 数据清洗
test['osv'] = test['osv'].apply(osv_trans)

f073b_changxiang_v01_b1b8_20180915#########
%E6%B1%9F%E7%81%B5OS+5.0#########
GIONEE_YNGA#########


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [6]:
#train['os'].value_counts()
train['lan'].value_counts()
#lan_map = {'zh-CN': 1, }
train['lan'].value_counts().index
lan_map = {'zh-CN': 1, 'zh_CN':2, 'Zh-CN': 3, 'zh-cn': 4, 'zh_CN_#Hans':5, 'zh': 6, 'ZH': 7, 'cn':8, 'CN':9, 'zh-HK': 10, 'tw': 11, 'TW': 12, 'zh-TW': 13,'zh-MO':14, 'en':15, 'en-GB': 16, 'en-US': 17, 'ko': 18, 'ja': 19, 'it': 20, 'mi':21} 
train['lan'] = train['lan'].map(lan_map)
test['lan'] = test['lan'].map(lan_map)
test['lan'].value_counts()

1.0     73046
6.0      9094
8.0      6136
2.0      3391
3.0      2430
4.0       165
7.0        60
9.0        46
11.0       28
15.0       26
5.0        11
18.0        6
13.0        5
17.0        2
10.0        2
12.0        1
16.0        1
Name: lan, dtype: int64

In [7]:
train['lan'].fillna(22, inplace=True)
test['lan'].fillna(22, inplace=True)


# In[7]:


remove_list = ['os', 'sid']
col = features
for i in remove_list:
    col.remove(i)
col

['android_id',
 'apptype',
 'carrier',
 'dev_height',
 'dev_ppi',
 'dev_width',
 'lan',
 'media_id',
 'ntt',
 'osv',
 'package',
 'timestamp',
 'version',
 'fea_hash',
 'location',
 'fea1_hash',
 'cus_type']

In [8]:
from datetime import datetime

# lambda 是一句话函数，匿名函数
train['timestamp'] = train['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))
#1559892728241.7212
#1559871800477.1477
#1625493942.538375
#import time
#time.time()
test['timestamp'] = test['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))
test['timestamp']

0        2019-06-07 09:43:20.477148
1        2019-06-05 20:53:33.449314
2        2019-06-04 10:08:24.732394
3        2019-06-05 01:03:10.643892
4        2019-06-05 08:16:34.450110
                    ...            
149995   2019-06-08 09:21:33.629823
149996   2019-06-07 07:15:57.431662
149997   2019-06-09 08:48:59.661749
149998   2019-06-05 06:38:14.527250
149999   2019-06-08 07:23:16.149025
Name: timestamp, Length: 150000, dtype: datetime64[ns]

In [9]:
def version_trans(x):
    if x=='V3':
        return 3
    if x=='v1':
        return 1
    if x=='P_Final_6':
        return 6
    if x=='V6':
        return 6
    if x=='GA3':
        return 3
    if x=='GA2':
        return 2
    if x=='V2':
        return 2
    if x=='50':
        return 5
    return int(x)
train['version'] = train['version'].apply(version_trans)
test['version'] = test['version'].apply(version_trans)
train['version'] = train['version'].astype('int')
test['version'] = test['version'].astype('int')

In [10]:
# 特征筛选
features = train[col]
# 构造fea_hash_len特征
features['fea_hash_len'] = features['fea_hash'].map(lambda x: len(str(x)))
features['fea1_hash_len'] = features['fea1_hash'].map(lambda x: len(str(x)))
# Thinking：为什么将很大的，很长的fea_hash化为0？
# 如果fea_hash很长，都归为0，否则为自己的本身
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features

test_features = test[col]
# 构造fea_hash_len特征
test_features['fea_hash_len'] = test_features['fea_hash'].map(lambda x: len(str(x)))
test_features['fea1_hash_len'] = test_features['fea1_hash'].map(lambda x: len(str(x)))
# Thinking：为什么将很大的，很长的fea_hash化为0？
# 如果fea_hash很长，都归为0，否则为自己的本身
test_features['fea_hash'] = test_features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features['fea1_hash'] = test_features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,fea_hash_len,fea1_hash_len
0,317625,1181,46000.0,2196.0,2.0,1080.0,9.0,639,2.0,8.10,188,2019-06-07 09:43:20.477148,7,1672223856,57,3872258917,658,10,10
1,435108,944,46003.0,2280.0,3.0,1080.0,1.0,704,6.0,8.10,221,2019-06-05 20:53:33.449314,3,3767901757,23,129322164,943,10,9
2,0,1106,46000.0,0.0,0.0,0.0,22.0,39,2.0,5.10,1562,2019-06-04 10:08:24.732394,0,454638703,30,4226678391,411,9,10
3,451504,761,46000.0,1344.0,0.0,720.0,22.0,54,2.0,7.11,9,2019-06-05 01:03:10.643892,0,1507622951,65,3355419572,848,10,10
4,0,1001,46000.0,665.0,0.0,320.0,1.0,29,5.0,8.10,4,2019-06-05 08:16:34.450110,0,4116351093,148,2644467751,411,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0,1001,46000.0,760.0,0.0,360.0,1.0,29,2.0,8.10,4,2019-06-08 09:21:33.629823,0,3162887451,126,2711576615,411,10,10
149996,0,1001,46000.0,780.0,0.0,360.0,1.0,29,2.0,9.00,4,2019-06-07 07:15:57.431662,0,97238959,322,2678022183,411,8,10
149997,0,1001,46000.0,780.0,0.0,360.0,1.0,29,5.0,8.10,4,2019-06-09 08:48:59.661749,0,1320118495,46,2610913319,411,10,10
149998,500925,1052,46000.0,854.0,240.0,480.0,1.0,249,6.0,4.42,0,2019-06-05 06:38:14.527250,2,1292986591,41,1898209327,430,10,10


In [11]:
# 对训练集的timestamp提取时间多尺度

# 使用to_datetime进行日期类型转换
# 创建时间戳索引
temp = pd.DatetimeIndex(features['timestamp'])
features['year'] = temp.year
features['month'] = temp.month
features['day'] = temp.day
features['week_day'] = temp.weekday #星期几
features['hour'] = temp.hour
features['minute'] = temp.minute

# 求时间的diff
start_time = features['timestamp'].min()
features['time_diff'] = features['timestamp'] - start_time
features['time_diff'] = features['time_diff'].dt.days + features['time_diff'].dt.seconds/3600/24
features[['timestamp', 'year', 'month', 'day', 'week_day', 'hour', 'minute', 'time_diff']]

# 创建时间戳索引
temp = pd.DatetimeIndex(test_features['timestamp'])
test_features['year'] = temp.year
test_features['month'] = temp.month
test_features['day'] = temp.day
test_features['week_day'] = temp.weekday #星期几
test_features['hour'] = temp.hour
test_features['minute'] = temp.minute

# 求时间的diff
#start_time = features['timestamp'].min()
test_features['time_diff'] = test_features['timestamp'] - start_time
test_features['time_diff'] = test_features['time_diff'].dt.days + test_features['time_diff'].dt.seconds/3600/24
#test_features[['timestamp', 'year', 'month', 'day', 'week_day', 'hour', 'minute', 'time_diff']]
test_features['time_diff']

0         4.405081
1         2.870509
2         1.422488
3         2.043854
4         2.344826
            ...   
149995    5.389954
149996    4.302731
149997    6.367338
149998    2.276539
149999    5.307801
Name: time_diff, Length: 150000, dtype: float64

In [12]:
#test['version'].value_counts()
#features['version'].value_counts()
features['dev_height'].value_counts()
features['dev_width'].value_counts()
# 构造面积特征
features['dev_area'] = features['dev_height'] * features['dev_width']
test_features['dev_area'] = test_features['dev_height'] * test_features['dev_width']


# In[13]:
#分辨率除以面积特征
features['ppivsarea'] = features['dev_ppi'].astype('float') / features['dev_area'].astype('float')
test_features['ppivsarea'] = test_features['dev_ppi'].astype('float') / test_features['dev_area'].astype('float')
"""
Thinking：是否可以利用 dev_ppi 和 dev_area构造新特征
features['dev_ppi'].value_counts()
features['dev_area'].astype('float') / features['dev_ppi'].astype('float')
"""
#features['ntt'].value_counts()
features['carrier'].value_counts()
features['package'].value_counts()
# version - osv APP版本与操作系统版本差
features['osv'].value_counts()
features['version_osv'] = features['osv'] - features['version']
test_features['version_osv'] = test_features['osv'] - test_features['version']


# In[14]:


features = features.drop(['timestamp'], axis=1)
test_features = test_features.drop(['timestamp'], axis=1)

In [13]:
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,1,22.0,104,6.0,android,9.0,18,1438873,2019-06-07 15:32:08.241721,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,1,22.0,19,6.0,android,8.1,0,1185582,2019-06-08 19:40:20.922229,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,1,22.0,559,0.0,android,8.1,0,1555716,2019-06-06 23:58:17.549109,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,0,22.0,129,2.0,android,8.1,0,1093419,2019-06-09 08:59:37.719685,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,1.0,64,2.0,android,8.0,0,1400089,2019-06-07 08:28:21.478553,5,2364522023,4,1510695983,582


In [14]:
#从train中找到selected字段的关键特征值把他设置出来
def find_key_feature(train,selected):
    temp0=train[train['label']==0]
    temp=pd.DataFrame(columns=[0,1])
    temp[0]=temp0[selected].value_counts()/len(temp0)*100

    temp1=train[train['label']==1]

    temp[1]=temp1[selected].value_counts()/len(temp1)*100
    temp[2]=temp[1]/temp[0]
    #筛选出特征比大于10倍的关键特征
    result=temp[temp[2]>10].sort_values(2,ascending=False).index
    return result
key_feature={}
key_feature['osv']=find_key_feature(train,'osv')
key_feature['dev_ppi']=find_key_feature(train,'dev_ppi')
key_feature['apptype']=find_key_feature(train,'apptype')
train['media_id']=features['media_id']
key_feature['media_id']=find_key_feature(train,'media_id')
key_feature

{'osv': Float64Index([   7.77,    7.21,    7.75,    7.85,    7.87,     3.8,    7.67,
                   3.9,     2.3,    8.01,     7.9,    3.84,    7.64,    7.89,
               21100.0,    7.92,     4.1,    7.72,    7.82,     7.8,    3.86,
                   7.7,    7.84,    7.68, 21000.0,    7.86,    7.73,     6.1,
                  3.83,    3.78,    7.74],
              dtype='float64'),
 'dev_ppi': Float64Index([200.0, 230.0, 128.0], dtype='float64'),
 'apptype': Int64Index([1139, 716, 941, 851, 1034, 1067], dtype='int64'),
 'media_id': Int64Index([ 329,  384,  259,  249,  334,  734,  504,  224,  614,  254,  304,
              899,  414,  954,  764,  654,   74, 1524,  449,  344,  324,  119,
               24, 1454],
            dtype='int64')}

In [15]:
features

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,...,year,month,day,week_day,hour,minute,time_diff,dev_area,ppivsarea,version_osv
0,316361,1199,46000.0,0.0,0.0,0.0,22.0,104,6.0,9.00,...,2019,6,7,4,15,32,4.647303,0.0,NaN,1.00
1,135939,893,0.0,0.0,0.0,0.0,22.0,19,6.0,8.10,...,2019,6,8,5,19,40,5.819664,0.0,NaN,4.10
2,399254,821,0.0,760.0,0.0,360.0,22.0,559,0.0,8.10,...,2019,6,6,3,23,58,3.998796,273600.0,0.000000,8.10
3,68983,1004,46000.0,2214.0,0.0,1080.0,22.0,129,2.0,8.10,...,2019,6,9,6,8,59,6.374722,2391120.0,0.000000,8.10
4,288999,1076,46000.0,2280.0,0.0,1080.0,1.0,64,2.0,8.00,...,2019,6,7,4,8,28,4.353009,2462400.0,0.000000,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,392477,1028,46000.0,1920.0,3.0,1080.0,1.0,144,6.0,7.12,...,2019,6,6,3,23,14,3.968252,2073600.0,0.000001,0.12
499996,346134,1001,0.0,1424.0,0.0,720.0,22.0,29,2.0,8.10,...,2019,6,6,3,17,40,3.736713,1025280.0,0.000000,5.10
499997,499635,761,46000.0,1280.0,0.0,720.0,22.0,54,6.0,6.01,...,2019,6,5,2,3,14,2.135370,921600.0,0.000000,6.01
499998,239786,917,46001.0,960.0,0.0,540.0,2.0,109,2.0,5.11,...,2019,6,7,4,0,59,4.041273,518400.0,0.000000,5.11


In [16]:
def f(x,selected):
    if x in key_feature[selected]:
        return 1
    else:
        return 0
features['osv1']=features['osv'].apply(f,args=('osv',))
test_features['osv1']=test_features['osv'].apply(f,args=('osv',))

features['dev_ppi1']=features['dev_ppi'].apply(f,args=('dev_ppi',))
test_features['dev_ppi1']=test_features['dev_ppi'].apply(f,args=('dev_ppi',))

# features['apptype1']=features['apptype'].apply(f,args=('apptype',))
# test_features['apptype1']=test_features['apptype'].apply(f,args=('apptype',))

# features['media_id']=features['media_id'].apply(f,args=('media_id',))
# test_features['media_id']=test_features['media_id'].apply(f,args=('media_id',))

# print(test_features['media_id'].value_counts())
# features['osv1'].value_counts()

In [17]:
features.columns

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'osv', 'package', 'version',
       'fea_hash', 'location', 'fea1_hash', 'cus_type', 'fea_hash_len',
       'fea1_hash_len', 'year', 'month', 'day', 'week_day', 'hour', 'minute',
       'time_diff', 'dev_area', 'ppivsarea', 'version_osv', 'osv1',
       'dev_ppi1'],
      dtype='object')

In [18]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
# skf = StratifiedKFold(n_splits=3, random_state=2021)
def xgb_model(clf,train_x,train_y,test):
    prob=[]
    mean_acc=0
    nums=5
    sk=StratifiedKFold(n_splits=nums, shuffle=True)
    for k,(train_index,val_index) in enumerate(sk.split(train_x,train_y)):
        train_x_real=train_x.iloc[train_index]
        train_y_real=train_y.iloc[train_index]
        clf.fit(train_x_real,train_y_real)
        val_x=train_x.iloc[val_index]
        val_y=train_y.iloc[val_index]
        val_y_pred=clf.predict(val_x)
        acc_val=accuracy_score(val_y,val_y_pred)
        print('第{}个模型的accuracy为{}'.format(k,acc_val))
        mean_acc+=acc_val/nums
        test_y_pred=clf.predict_proba(test)
        prob.append(test_y_pred)
        
    print(mean_acc)
    mean_prob=sum(prob)/nums
    print(prob)
    return mean_prob

In [19]:
# #这个分数是89.0627，并且屏蔽了关键点判断
# import lightgbm as lgb
# import xgboost as xgb
# # model = lgb.LGBMClassifier()
# xgb = xgb.XGBClassifier(
#             max_depth=15, learning_rate=0.005, n_estimators=3000, 
#             objective='binary:logistic', tree_method='gpu_hist', 
#             subsample=0.7, colsample_bytree=0.7, 
#             min_child_samples=3, eval_metric='auc', reg_lambda=0.5
#         )

# prob=xgb_model(xgb,features,train['label'],test_features)

In [20]:
features['heightvswidth'] = features['dev_height'].astype('float') / features['dev_width'].astype('float')
test_features['heightvswidth'] = test_features['dev_height'].astype('float') / test_features['dev_width'].astype('float')

In [21]:
#跑到89.132分用了关键点提取，5折交叉验证
import lightgbm as lgb
import xgboost as xgb
# model = lgb.LGBMClassifier()
xgb = xgb.XGBClassifier(
            max_depth=15, learning_rate=0.005, n_estimators=3000, 
            objective='binary:logistic', tree_method='gpu_hist', 
            subsample=0.7, colsample_bytree=0.7, 
            min_child_samples=3, eval_metric='auc', reg_lambda=0.5
        )

prob=xgb_model(xgb,features,train['label'],test_features)

[17:08:55] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [ ]:
# #再试试又加了一个app的关键点效果不好
# import lightgbm as lgb
# import xgboost as xgb
# # model = lgb.LGBMClassifier()
# xgb = xgb.XGBClassifier(
#             max_depth=15, learning_rate=0.005, n_estimators=3000, 
#             objective='binary:logistic', tree_method='gpu_hist', 
#             subsample=0.7, colsample_bytree=0.7, 
#             min_child_samples=3, eval_metric='auc', reg_lambda=0.5
#         )

# prob=xgb_model(xgb,features,train['label'],test_features)

In [ ]:
result=np.argmax(prob,axis=1)
result

In [ ]:
#features['version'].value_counts()
res = pd.DataFrame(test['sid'])
res['label'] = result
res.to_csv('./basel6++deep001.csv', index=False)
res